# Part I. ETL Pipeline for Pre-Processing the Files

### Import Python packages 

In [50]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


###  Creating list of filepaths to process original event csv data files

In [51]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


###  Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [52]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [53]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

 Creating a Cluster
 -------------------

In [54]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

 Create Keyspace
 -------------------

In [10]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace


In [55]:

session.set_keyspace('udacity')


### Query 1:

> *Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4*

Required fields: artist, song_title, length, sessionid, itemInSession

Partition Key: should be `sessionId`, because this is the primary way our data is being filtered in the query. We are also filtering on `itemInSession`, but that field is less specific and would result in a less logical distribution across nodes.

Primary Key: `sessionId` is not sufficient for a PK, because it is not unique. However, adding `itemInSession` as a clustering column with give us uniqueness.

In [15]:

drop_sessions="drop table IF EXISTS sessions"
session.execute(drop_sessions)

create_sessions="""
CREATE TABLE IF NOT EXISTS sessions
        (
        sessionId int, 
        itemInSession int, 
        artist text, 
        song_title text, 
        length float, 
        PRIMARY KEY(sessionId, itemInSession)
        )
"""

session.execute(create_sessions)



In [18]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Perform a SELECT to verify that the data have been inserted into each table

In [19]:
select_session_quert = "select artist, song_title, length from sessions WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(select_session_quert)
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2:

> *Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

Required fields: artist, song_title, itemInSession, first_name, last_name, userId, sessionId

Partition Key: should be `userId` in this case, because this is the primary way our data is being filtered in the query. We are also filtering on `sessionId`, but again that field is less specific and would result in a less logical distribution across nodes.

Primary Key: In this table `userId` would not be unique, and adding `sessionId` would still not guarantee uniqueness. However, the query also asks for sorting by `itemInSession`, which means we need it as a clustering column anyway. So a PK that is both unique and sorts the way we would like is `(userId, sessionId, itemInSession)`.

In [46]:

drop_users="drop table IF EXISTS users"
session.execute(drop_users)

create_users="""
    CREATE TABLE IF NOT EXISTS users(
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song_title text,
        first_name text,
        last_name text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
"""

session.execute(create_users)


In [47]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Perform a SELECT to verify that the data have been inserted into each table

In [48]:
select_session_quert = "select artist, song_title, first_name, last_name FROM users WHERE sessionId = 182 and userId = 10"

rows = session.execute(select_session_quert)
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3:

> *Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

Required fields: first_name, last_name, song_title, userId

Partition Key: we're filtering by `song_title` in this case, so we'll use that as our partition key

Primary Key: It's not necessarily required given our query, but I've chosen to include the `userId` field so that we can add it to our PK. I've done this for two reasons, the first being that we need a unique PK and `(song_title)` or `(song_title, firstName)` don't suffice. Another reason would be to give us some sorting by `userId`, which feels like a logical way to order. If instead we wanted to sort alphabetically we could do `(song_title, first_name, userId)`.

In [49]:
drop_song_users="drop table IF EXISTS song_users"
session.execute(drop_song_users)

create_song_users="""
    CREATE TABLE IF NOT EXISTS song_users (
        song_title text,
        userId int,
        first_name text,
        last_name text,
        PRIMARY KEY (song_title, userId)
    )
"""

session.execute(create_song_users)



In [42]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (song_title, userId, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))



#### Perform a SELECT to verify that the data have been inserted into each table

In [60]:
select_session_quert = "select first_name, last_name FROM song_users WHERE song_title='All Hands Against His Own'"

rows = session.execute(select_session_quert)

for row in rows:
    print (row.first_name, row.last_name) 

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions


In [43]:
drop_sessions="drop table IF EXISTS sessions"
session.execute(drop_sessions)

drop_users="drop table IF EXISTS users"
session.execute(drop_users)

drop_song_users="drop table IF EXISTS song_users"
session.execute(drop_song_users)



### Close the session and cluster connection


In [34]:
session.shutdown()
cluster.shutdown()